## AuxTel Image elongation due to azimuth oscillation

Craig Lage 17-Mar-22

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
EfdClient.list_efd_names()

In [ ]:
dir(client)

In [ ]:
topics = await client.get_topics()
for topic in topics:
    if 'exapo' in topic:
        print(topic)

In [ ]:
start = Time("2024-01-10T00:00:00", scale='utc')
end = Time("2024-01-10T01:00:00", scale='utc')
el = await client.select_time_series("lsst.sal.MTMount.elevation", \
                                            ['actualPosition'], \
                                             start, end)


In [ ]:
M1M3_forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", "*", \
                                         start, end)

In [ ]:
M1M3_forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", ["zForce0"], \
                                         start, end)

In [ ]:
from lsst.ts.xml.tables.m1m3 import FATable
M1M3_names = []
for i in range(len(FATable)):
    name=f"zForce{i}"
    M1M3_names.append(name)
for i in range(len(FATable)):
    if FATable[i].actuator_type.name == 'DAA':
        if FATable[i].orientation.name in ['X_PLUS', 'X_MINUS']:
            name = f"xForce{FATable[i].x_index}"
            M1M3_names.append(name)
        if FATable[i].orientation.name in ['Y_PLUS', 'Y_MINUS']:
            name = f"yForce{FATable[i].y_index}"
            M1M3_names.append(name)
print(len(names))

In [ ]:
M1M3_forces.columns

In [ ]:
# This is almost 4 times faster.
M1M3_forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", 
                                              names, 
                                         start, end)

In [ ]:
M1M3_forces.columns

In [ ]:
el.index[0]

In [ ]:
(el.index[10] - el.index[0])/10

In [ ]:
3600/7.6

In [ ]:
start = Time("2024-05-28 18:00:00Z", scale='utc')
end = Time("2024-05-29 12:00:00Z", scale='utc')
elec = await client.select_time_series('lsst.sal.Electrometer.logevent_intensity', ['intensity'], start, end, index=201)
print(f"There are {len(elec)} measurements")
elec['intensity'].plot(marker='x')

In [ ]:
elec['intensity'].plot(marker='x')


In [ ]:
pos = await client.select_time_series('lsst.sal.PMD.position', ['*'], start, end)
len(pos)

In [ ]:
pos.columns

In [ ]:
[np.isnan(x) for x in pos['position0'].values if np.isnan(x)]


In [ ]:
pos.head(5)

In [ ]:
pos.tail(5)

In [ ]:
start = Time("2024-05-29 20:30:00Z", scale='utc')
end = Time("2024-05-29 21:00:00Z", scale='utc')
pos_now = await client.select_time_series('lsst.sal.PMD.position', ['*'], start, end)
len(pos_now)

In [ ]:
pos_now.head(3)

In [ ]:
fig, axs = plt.subplots(3,2,figsize=(10,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle("AuxTel M1 Position micrometers - 2024", fontsize=24)
counter = 0
for i in range(3):
    for j in range(2):
        axs[i][j].set_title(f"position{counter}")
        if counter == 4:
            counter += 1
            continue
        pos[f'position{counter}'].plot(ax = axs[i][j],marker='x', label="2024-03-19")
        axs[i][j].axhline(np.mean(pos_now[f'position{counter}'].values), ls='--', color='red', label="Current Measurements")
        counter += 1
        axs[i][j].legend()
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_graphs/M1_Micrometers_2024.png")  

In [ ]:
camhex = await client.select_time_series('lsst.sal.MTHexapod.actuators', ['*'], start, end, index=1)
print(len(camhex))
m2hex = await client.select_time_series('lsst.sal.MTHexapod.actuators', ['*'], start, end, index=2)
print(len(m2hex))

In [ ]:
print(m2hex.columns)
print(camhex.columns)

In [ ]:
fig, axs = plt.subplots(6,2, figsize = (5,12))
plt.subplots_adjust(hspace=1.0, wspace=0.5)
names = ['Cam', 'M2']
for i, hex in enumerate([camhex, m2hex]):
    for j in range(6):
        axs[j][i].set_title(f"{names[i]} - calibrated{j}")
        hex[f"calibrated{j}"].plot(ax = axs[j][i])

In [ ]:
# Times to start looking at hexapods
start = Time("2024-05-01 17:00:00Z", scale='utc')
end = Time("2024-05-01 17:10:00Z", scale='utc')


In [ ]:
camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
print(len(camhex))
m2hex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=2)
print(len(m2hex))

In [ ]:
print(m2hex.columns)
print(camhex.columns)

In [ ]:
fig, axs = plt.subplots(6,2, figsize = (5,12))
plt.subplots_adjust(hspace=1.0, wspace=0.5)
hexNames = ['Cam', 'M2']
posNames = ['X','Y','Z','U','V','W']
for i, hex in enumerate([camhex, m2hex]):
    for j in range(6):
        axs[j][i].set_title(f"{hexNames[i]} - {posNames[j]}")
        hex[f"position{j}"].plot(ax = axs[j][i])

In [ ]:
for i in range(len(state)):
    print(Time(state.index[i]).isot, state['detailedState'][i])


In [ ]:
gyro = await client.select_time_series("lsst.sal.MTM1M3.gyroData", ['*'], start, end)

In [ ]:
len(gyro)

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  start, end)

In [ ]:
len(az)

In [ ]:
times = az.values[:,1]
azs = az.values[:,0]

In [ ]:
for i in range(1, len(times)):
    print(i, (times[i] - times[i-1]), (azs[i] - azs[i-1]))

In [ ]:
# Times to start looking at error codes
start = Time("2023-11-08 06:49:33Z", scale='utc')
end = Time("2023-11-08 06:49:39Z", scale='utc')

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  start, end)

In [ ]:
len(az)

In [ ]:
times = az.values[:,1]
azs = az.values[:,0]

In [ ]:
for i in range(1, len(times)):
    print(i, f"DeltaT = {(times[i] - times[i-1])}")#, (azs[i] - azs[i-1])")

In [ ]:
# Times to start looking at error codes
start = Time("2023-11-08 06:18:50Z", scale='utc')
end = Time("2023-11-08 06:18:55Z", scale='utc')

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  start, end)

In [ ]:
len(az)

In [ ]:
times = az.values[:,1]
azs = az.values[:,0]

In [ ]:
for i in range(1, len(times)):
    print(i, (times[i] - times[i-1]), (azs[i] - azs[i-1]))

In [ ]:
az["azimuthCalculatedAngle"].plot(marker='x')

In [ ]:
client = EfdClient('idf_efd')

In [ ]:
# Times to start looking at error codes
start = Time("2023-05-05T00:00:00Z", scale='utc')
end = Time("2023-05-06T00:00:00Z", scale='utc')
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  start, end)
print(len(az))

In [ ]:
end

In [ ]:
az.tail(2)

In [ ]:
az['azimuthCalculatedAngle'].plot()

In [ ]:
len(az)

In [ ]:
times = az.values[:,1]
azs = az.values[:,0]

In [ ]:
for i in range(1, len(times)):
    print(i, f"DeltaT = {(times[i] - times[i-1])}")#, (azs[i] - azs[i-1])")

In [ ]:
# Times to start looking at error codes
start = Time("2023-11-08 06:50:20Z", scale='utc')
end = Time("2023-11-08 06:50:45Z", scale='utc')

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  start, end)

In [ ]:
len(az)

In [ ]:
times = az.values[:,1]
azs = az.values[:,0]

In [ ]:
for i in range(1, len(times)):
    print(i, f"DeltaT = {(times[i] - times[i-1])}")#, (azs[i] - azs[i-1])")

In [ ]:
plt.plot(times[0:100], azs[0:100], marker='x')

In [ ]:
az["azimuthCalculatedAngle"].plot(marker='x')
p1 = Time("2023-11-08 06:50:39.8Z", scale='utc').isot
p2 = Time("2023-11-08 06:50:39.9Z", scale='utc').isot
plt.xlim(p1,p2)

In [ ]:
az["azimuthCalculatedAngle"].head(80)

In [ ]:
# Times to start looking at error codes
start = Time("2023-07-11 12:00:00Z", scale='utc')
end = Time("2023-07-12 12:00:00Z", scale='utc')

In [ ]:
test = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                ['actualPosition'],  start, end)

In [ ]:
len(test)

In [ ]:
test['maxSpeed'].plot()
plt.title('lsst.sal.ESS.airFlow.maxSpeed')
plt.ylabel('m/s')

In [ ]:
# Look before and after the last error time
lastErrorTime = Time(errors.index[3])
before = 300.0
after = 300.0
start = lastErrorTime - TimeDelta(before, format='sec')
end = lastErrorTime + TimeDelta(after, format='sec')
print(start, end)

In [ ]:
status = await client.select_time_series('lsst.sal.ATMCS.logevent_azimuthDrive1Status', \
                                                ['*'],  start, end)

In [ ]:
len(status)

In [ ]:
status

In [ ]:
# Times to start looking at error codes
start = Time("2022-03-17 00:00:00Z", scale='utc')
end = Time("2022-03-17 00:05:00Z", scale='utc')
test = await client.select_packed_time_series('lsst.sal.ATPtg.mountPositions', \
                                                ['*'],  start, end)

In [ ]:
start = Time("2023-06-01T00:00:00Z", scale='utc')
end = Time("2023-06-01T11:00:00Z", scale='utc')
test = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthMotionState', \
                                                ['*'],  start, end)

In [ ]:
len(test)

In [ ]:
# Times to start looking at error codes
start = Time("2023-11-08 06:45:00Z", scale='utc')
end = Time("2023-11-08 06:55:00Z", scale='utc')

In [ ]:
cRIO_ts = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ["cRIO_timestamp"],  start, end)

In [ ]:
len(cRIO_ts)

In [ ]:
timestamps = cRIO_ts.values

In [ ]:
for i in range(1, len(timestamps)):
    print(i, timestamps[i] - timestamps[i-1])

In [ ]:
clientclient = EfdClient('idf_efd')

In [ ]:
# Times to start looking at error codes
start = Time("2024-02-16 19:45:00Z", scale='utc')
end = Time("2024-02-16 20:00:00Z", scale='utc')

In [ ]:
start = Time("2024-02-16 19:45:00Z", scale='utc')
end = Time("2024-02-16 20:00:00Z", scale='utc')
gill = await client.select_time_series("lsst.sal.ESS.airTurbulence", \
                                            ["*"],  start, end, index=205)

In [ ]:
gill

In [ ]:
timestamps = cRIO_ts.values

In [ ]:
timestamps = cRIO_ts.values
for i in range(1, len(timestamps)):
    print(i, timestamps[i] - timestamps[i-1])

In [ ]:
client = EfdClient('usdf_efd')
times = [["2023-09-22 03:45:00Z", "2023-09-22 03:55:00Z"], ["2023-11-08 06:45:00Z", "2023-11-08 06:55:00Z"], \
        ["2023-11-08 08:20:00Z", "2023-11-08 08:30:00Z"]]

image_names = [None, 2023110700519, 2023110700594]

image_stamps = [None, pd.Timestamp("2023-11-08 06:50:43Z", tz='UTC'), pd.Timestamp("2023-11-08 08:26:20Z", tz='UTC')]
fig, axs = plt.subplots(3,1,figsize=(8, 12))
#plt.subplots_adjust(hspace= 0.5)
for i, [tstart, tend] in enumerate(times):
    start = Time(tstart, scale='utc')
    end = Time(tend, scale='utc')

    cRIO_ts = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                                ["cRIO_timestamp"],  start, end)
    print(tstart, len(cRIO_ts))
    timestamps = cRIO_ts.values
    index = cRIO_ts.index
    xaxis = []
    deltaTs = []
    for n in range(1, len(timestamps)):
        xaxis.append(index[n])
        deltaTs.append(timestamps[n] - timestamps[n-1])
    sigma_deltaT = np.std(deltaTs)
    axs[i].set_title(f"cRIO timestamp deltas {tstart}")
    axs[i].plot(xaxis, deltaTs)
    axs[i].set_ylim(0.9, 1.35)
    axs[i].set_ylabel("cRIO timestamp delta (sec)")
    axs[i].text(xaxis[1], 1.2, f"deltaT_sigma = {sigma_deltaT*1E6:.2f} microsec")
    if image_names[i] is not None:
        axs[i].plot([image_stamps[i], image_stamps[i]], [0.9,1.35], ls = '--', color='red')
        axs[i].text(image_stamps[i], 0.95, f"{image_names[i]}")
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_graphs/cRIO_Timestamps_2_15Nov23.png")                 

In [ ]:
client = EfdClient('usdf_efd')
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections="LATISS/raw/all")
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
fig, axs = plt.subplots(1,2,figsize=(10,5))

expIdList = [2023110700489, 2023110700527]
for i, expId in enumerate(expIdList):
    dataId = {'detector':0, 'exposure':expId}
    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    t_start = Time(tStart, scale='tai').utc
    t_end = Time(tEnd, scale='tai').utc
    cRIO_ts = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                                ["cRIO_timestamp"],  t_start, t_end)
                                                
    timestamps = cRIO_ts.values
    index = cRIO_ts.index
    xaxis = []
    deltaTs = []
    for n in range(1, len(timestamps)):
        xaxis.append(index[n])
        deltaTs.append(timestamps[n] - timestamps[n-1])
    sigma_deltaT = np.std(deltaTs)
    axs[i].set_title(f"cRIO timestamp deltas {expId}")
    axs[i].plot(xaxis, deltaTs)
    axs[i].set_ylim(0.9, 1.35)
    axs[i].set_ylabel("cRIO timestamp delta (sec)")
    axs[i].text(xaxis[1], 1.2, f"deltaT_sigma = {sigma_deltaT*1E6:.2f} microsec")
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_classifier/cRIO_Timestamps_02Jan24.png")                 

In [ ]:
type(xaxis[1])

In [ ]:
plt.plot(xaxis, deltaTs)
plt.ylim(0.90, 1.35)

In [ ]:
cRIO_ts.index

In [ ]:
# Times to start looking at error codes
start = Time("2023-05-01T00:00:00Z", scale='utc')
end = Time("2024-02-01T00:00:00Z", scale='utc')
tempCCD = await client.select_time_series("lsst.sal.ATCamera.vacuum", \
                                            "tempCCD",  start, end)

In [ ]:
len(az)

In [ ]:
rolling = tempCCD['tempCCD'].rolling(10).sum() / 10.0
#tempCCD['tempCCD'].plot()
rolling.plot()
plt.title("AuxTel CCD temperature")
plt.ylim(-93.6, -94.6)
plt.ylabel("Temp (C)")
plt.axhline(-94.15, ls = '--', color='black', label="Setpoint")
plt.savefig("/home/c/cslage/u/AuxTel/gains/AuxTel_CCD_Temp_2023_2024.png")

In [ ]:
# Times to start looking
start = Time("2024-04-23 23:00:00Z", scale='utc')
end = Time("2024-04-23 23:50:00Z", scale='utc')
vels = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", \
                                                ["azimuthMotor1Velocity", "azimuthMotor2Velocity"],  start, end)

In [ ]:
len(vels)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.subplots_adjust(wspace=0.3)
vels["azimuthMotor1Velocity"].plot(ax=axs[0], color='blue', label='Motor1')
vels["azimuthMotor2Velocity"].plot(ax=axs[0], color='red', ls = '--', label='Motor2')
vels["azimuthMotor1Velocity"].plot(ax=axs[1], color='blue', label='Motor1')
vels["azimuthMotor2Velocity"].plot(ax=axs[1], color='red', ls = '--', label='Motor2')
axs[0].legend()
axs[0].set_ylabel("Velocity(deg/s)")
axs[1].set_ylim(-0.02, 0.02)
plotStart = Time("2024-04-23 23:23:00Z", scale='utc')
plotEnd = Time("2024-04-23 23:23:50Z", scale='utc')
axs[1].set_xlim(plotStart.isot, plotEnd.isot)
axs[1].legend()
axs[1].set_ylabel("Velocity(deg/s)")
plt.savefig("/home/c/cslage/u/AuxTel/mount_graphs/Azimuth_Velocity_Check_23Apr24.png")